In [2]:
import ee
import geopandas
import numpy as np
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium
from shapely.geometry import box
import warnings
from functools import reduce
from math import sin, cos, sqrt, atan2, radians

In [3]:
ee.Initialize()

Create a dictionary of countries with its boundary coordinates

In [4]:
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
country_polygon_dict = dict()
for index, row in world.iterrows():
    if 'MultiPolygon' in str(type(row['geometry'])):
        i = 0
        for poly in row['geometry']:
            # Remove small area that's not needed for sampling
            if poly.area > 5:
                # Area is in the unit of squre degrees
                country_polygon_dict[row['iso_a3']+'_'+str(i)] = (poly, round(poly.area*0.2))
                i +=1 
    # Here implies a Polygon
    elif row['geometry'].area > 5:
        country_polygon_dict[row['iso_a3']] = (row['geometry'], round(row['geometry'].area*0.2))

In [5]:
# Remove miscellaneous countries
del country_polygon_dict['-99']
del country_polygon_dict['-99_0']
del country_polygon_dict['-99_1']
del country_polygon_dict['-99_2']

In [6]:
country_polygon_dict

{'AFG': (<shapely.geometry.polygon.Polygon at 0x7f81d7c37320>, 13),
 'AGO_0': (<shapely.geometry.polygon.Polygon at 0x7f81d7c35cf8>, 21),
 'ARE': (<shapely.geometry.polygon.Polygon at 0x7f81d7c2e748>, 1),
 'ARG_0': (<shapely.geometry.polygon.Polygon at 0x7f81d7c351d0>, 55),
 'ATA_0': (<shapely.geometry.polygon.Polygon at 0x7f81d7c2d908>, 4),
 'ATA_1': (<shapely.geometry.polygon.Polygon at 0x7f81d7c2db00>, 3),
 'ATA_2': (<shapely.geometry.polygon.Polygon at 0x7f81d7c2da90>, 1197),
 'AUS_0': (<shapely.geometry.polygon.Polygon at 0x7f81d7c26940>, 1),
 'AUS_1': (<shapely.geometry.polygon.Polygon at 0x7f81d7c26ac8>, 138),
 'AUT': (<shapely.geometry.polygon.Polygon at 0x7f81d7c38390>, 2),
 'AZE_0': (<shapely.geometry.polygon.Polygon at 0x7f81d7c2d0b8>, 2),
 'BEN': (<shapely.geometry.polygon.Polygon at 0x7f81d7c35b38>, 2),
 'BFA': (<shapely.geometry.polygon.Polygon at 0x7f81d7c26b38>, 5),
 'BGD': (<shapely.geometry.polygon.Polygon at 0x7f81d7c2fd30>, 2),
 'BGR': (<shapely.geometry.polygon.Pol

In [ ]:
https://gis.stackexchange.com/questions/6412/generate-points-that-lie-inside-polygon
    
    
https://stackoverflow.com/questions/58802921/using-geopandas-how-to-randomly-select-in-each-polygon-5-points-by-sampling-met
    
    
https://developers.google.com/earth-engine/geometries

Pull Sentinel-2 Data

In [41]:
def satellite_imagery_Sentinel(area_of_interest_shapely,
                      start_date, end_date, 
                      plot_option, resolution=30):
    
    area_of_interest_ee = ee.Geometry.Polygon(list(area_of_interest_shapely.exterior.coords))

    
    # Create image collection that contains the area of interest
    img_collect = (ee.ImageCollection('COPERNICUS/S2')
                 .filterDate(start_date, end_date)
                 .filterBounds(area_of_interest_ee)
                    # Remove image that's too small (likely to be partial image)
                    # Size of a full image: 1,276,131,371; size of a partial image: 276,598,191
                 .filter(ee.Filter.gt('system:asset_size', 800000000))
                 .filterMetadata("CLOUDY_PIXEL_PERCENTAGE","less_than",50))
    
#         img_collect = img_collect.filterMetadata("CLOUDY_PIXEL_PERCENTAGE","less_than",20) 
    
    assert (img_collect.size().getInfo()>0), "No valid image"
    print("Total number of images in the collection: ", img_collect.size().getInfo())
        
    # Extract tile information from each image
    # Note: tiles can overlap a little bit
    unique_tiles = set([item['properties']['MGRS_TILE'] for item in img_collect.getInfo()['features']])
    if len(unique_tiles) > 1:
        warnings.warn('Multiple tiles are selected. Proceed with caution.')
        print('Number of tiles selected: ', len(unique_tiles))
#     if img_collect_no_partial.size().getInfo() < img_collect.size().getInfo():
#         warnings.warn('There are partial images in the collection. Proceed with caution.')
#         print('Number of partial images: ', img_collect.size().getInfo()-img_collect_no_partial.size().getInfo())
        
    # Reference: https://www.satimagingcorp.com/satellite-sensors/other-satellite-sensors/sentinel-2a/
    band_blue = 'B2' #10m
    band_green = 'B3' #10m
    band_red = "B4"  #10m
    band_nir = 'B8'  #10m
    
    def calc_NDVI(img):
        ndvi = ee.Image(img.normalizedDifference([band_nir, band_red])).rename(["ndvi"]).copyProperties(img, img.propertyNames())
        composite = img.addBands(ndvi)
        return composite
    
    # SAVI = ((NIR – Red) / (NIR + Red + L)) x (1 + L)
    def calc_SAVI(img):
        """A function to compute Soil Adjusted Vegetation Index."""
        savi =  ee.Image(img.expression(
            '(1 + L) * float(nir - red)/ (nir + red + L)',
            {
                'nir': img.select(band_nir),
                'red': img.select(band_red),
                'L': 0.5
            })).rename(["savi"]).copyProperties(img, img.propertyNames())
        composite = img.addBands(savi)
        return composite

    # EVI = 2.5 * ((NIR – Red) / ((NIR) + (C1 * Red) – (C2 * Blue) + L))
    #     C1=6, C2=7.5, and L=1
    def calc_EVI(img):
        """A function to compute Soil Adjusted Vegetation Index."""
        evi = ee.Image(img.expression(
          '(2.5) * float(nir - red)/ ((nir) + (C1*red) - (C2*blue) + L)',
          {   
              'nir': img.select(band_nir),
              'red': img.select(band_red),
              'blue': img.select(band_blue),
              'L': 0.2,
              'C1': 6,
              'C2': 7.5
          })).rename(["evi"]).copyProperties(img, img.propertyNames())
        composite = img.addBands(evi)
        return composite
                   
    def add_landcover(img):
        landcover = ee.Image("USGS/GFSAD1000_V1")
        composite = img.addBands(landcover)
        return composite
    
    def calc_YYYYMM(img):
        return img.set('YYYYMM', img.date().format("YYYYMM"))
    
    def add_ee_layer(self, ee_object, vis_params, name):
        try:    
            if isinstance(ee_object, ee.image.Image):    
                map_id_dict = ee.Image(ee_object).getMapId(vis_params)
                folium.raster_layers.TileLayer(
                    tiles = map_id_dict['tile_fetcher'].url_format,
                    attr = 'Google Earth Engine',
                    name = name,
                    overlay = True,
                    control = True
                    ).add_to(self)
            elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
                ee_object_new = ee_object.median()
                map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
                folium.raster_layers.TileLayer(
                    tiles = map_id_dict['tile_fetcher'].url_format,
                    attr = 'Google Earth Engine',
                    name = name,
                    overlay = True,
                    control = True
                    ).add_to(self)
            elif isinstance(ee_object, ee.geometry.Geometry):    
                folium.GeoJson(
                        data = ee_object.getInfo(),
                        name = name,
                        overlay = True,
                        control = True
                    ).add_to(self)
            elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
                ee_object_new = ee.Image().paint(ee_object, 0, 2)
                map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
                folium.raster_layers.TileLayer(
                        tiles = map_id_dict['tile_fetcher'].url_format,
                        attr = 'Google Earth Engine',
                        name = name,
                        overlay = True,
                        control = True
                    ).add_to(self)

        except:
            print("Could not display {}".format(name))

    # Add EE drawing method to folium.
    folium.Map.add_ee_layer = add_ee_layer
    
    img_collect_calc = img_collect.map(calc_YYYYMM).map(calc_NDVI).map(calc_SAVI).map(calc_EVI).map(add_landcover)
    
    unique_month = list(set([item['properties']['YYYYMM'] for item in img_collect_calc.getInfo()['features']]))
    unique_month.sort()
    print(unique_month)
    
    if len(unique_month) > 0:
        warnings.warn('There are null values in the output DataFrame. Proceed with caution.')
    
#     1. min_lat, min_lon, max_lat, max_lon for the country
#             min_lon, min_lat, max_lon, max_lat = area_of_interest_shapely.bounds
#     2. from left to right, top to bottom, define 0.1 degree by 0.1 degree overlapping rectangles
#     3. only keep the rectangles that's within the country's boundary (possbily using spatial join)
#     4. for rectangle in a list of recntagles:
#         .reduceRegion(geometry=rectangle)
#         save to csv
    
    
    img_calc_month_dict = dict()
    temp_dict = dict()
    for month in unique_month:
        img_calc_month_dict[month] = img_collect_calc.filter(ee.Filter.eq('YYYYMM',month))
#         img_calc_month2 = img_calc_month_dict[month].addBands(ee.Image.pixelLonLat())
#         # EEException: Output of image computation is too large (20 bands for 851968 pixels = 126.8 MiB > 80.0 MiB).
#         #     If this is a reduction, try specifying a larger 'tileScale' parameter.
#         # EEException: ReduceRegion.AggregationContainer: Valid tileScales are 1 to 16.
#         data_month_lst = img_calc_month2.reduceRegion(reducer=ee.Reducer.toList(), \
#                                                              geometry=area_of_interest_ee, maxPixels=1e13, scale=resolution)

#         lat_series = pd.Series(np.array((ee.Array(data_month_lst.get("latitude")).getInfo())), name="lat")
#         lon_series = pd.Series(np.array((ee.Array(data_month_lst.get("longitude")).getInfo())), name="lon")
#         ndvi_series = pd.Series(np.array((ee.Array(data_month_lst.get("ndvi")).getInfo())), name=month+'_NDVI')
#         savi_series = pd.Series(np.array((ee.Array(data_month_lst.get("savi")).getInfo())), name=month+'_SAVI')
#         evi_series = pd.Series(np.array((ee.Array(data_month_lst.get("evi")).getInfo())), name=month+'_EVI')
#         temp_dict[month] = pd.concat([lat_series, lon_series, ndvi_series, savi_series, evi_series], axis=1)
    
#     df_lst = list(temp_dict.values())
#     out_df = reduce(lambda left, right: pd.merge(left,right,on=['lat', 'lon']), df_lst)

#     # Output the column names that have null values
#     if len(out_df.columns[out_df.isnull().any()]) > 0:
#         warnings.warn('There are null values in the output DataFrame. Proceed with caution.')

    # Create a folium map object.
    center_lon, center_lat = list(area_of_interest_shapely.centroid.coords)[0]
    myMap = folium.Map(location=[center_lat, center_lon], zoom_start=5)
    # Add the box around the area of interest
    folium.GeoJson(area_of_interest_shapely, name="Area of Interest").add_to(myMap)

    visParams = {'min':0, 'max':1, 'palette': ['red', 'yellow', 'green']}
    if plot_option == 'NDVI':
        for month in unique_month:
            myMap.add_ee_layer(img_calc_month_dict[month].select("ndvi"), visParams, name=plot_option+' '+month)
    elif plot_option == 'SAVI':
        for month in unique_month:
            myMap.add_ee_layer(img_calc_month_dict[month].select("savi"), visParams, name=plot_option+' '+month)
    elif plot_option == 'EVI':
        for month in unique_month:
            myMap.add_ee_layer(img_calc_month_dict[month].select("evi"), visParams, name=plot_option+' '+month)

    # Add a layer control panel to the map.
    myMap.add_child(folium.LayerControl())

#     return myMap, out_df
    return myMap

In [42]:
India_1_lat, India_1_lon = 23.967052, 72.400000
India_1_edge_len = 0.005

In [43]:
satellite_imagery_Sentinel(area_of_interest_shapely=country_polygon_dict['ARG_0'][0],
                  start_date='2018-1-01', end_date='2018-3-31', plot_option='NDVI')

Total number of images in the collection:  4893


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:26: UserWarning: Multiple tiles are selected. Proceed with caution.


Number of tiles selected:  375
['201801', '201802', '201803']


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:133: UserWarning: There are null values in the output DataFrame. Proceed with caution.


In [46]:
min_lon, min_lat, max_lon, max_lat = country_polygon_dict['ARG_0'][0].bounds

In [51]:
from shapely.geometry import Point
def random_point_in_shp(shp):
    within = False
    while not within:
        min_lon, min_lat, max_lon, max_lat = shp.bounds
        lon = np.random.uniform(min_lon, max_lon)
        lat = np.random.uniform(min_lat, max_lat)
        within = shp.contains(Point(lon,lat))
    return Point(lon,lat)

In [53]:
for num in range(5):
    print(random_point_in_shp(country_polygon_dict['ARG_0'][0]))

POINT (-60.68306322425825 -38.32631823832803)
POINT (-58.79757609729849 -37.75506217019666)
POINT (-64.72686559467584 -40.47010451451195)
POINT (-68.05127067102345 -24.39186781260426)
POINT (-66.37745332360406 -42.31355864041692)
